#### Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 200, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [5]:
embeddings = OllamaEmbeddings(model = "llama3.2")
db = FAISS.from_documents(docs, embeddings)
db

In [7]:
# querying

#query = "What does the speaker belive is the main reason the United States should enter the war?"
query = "How does the speaker describe the desired outcome of the war"
docs = db.similarity_search(query)
docs

[Document(id='7e296df7-a8db-4a0d-8fda-999f4c5e09c6', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8c0fc355-5d1f-4c44-bf68-971a785c4da5', metadata={'source': 'speech.txt'}, page_content='…'),
 Document(id='ab17ead1-248f-45b3-9553-100fdceb9887', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German bi

In [8]:
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

#### As a Retriever
We can also convert the vectorstore into a Retriver class. This allows us to easily use it in other LangChain moethods. which largely work with retrievers.

In [10]:
retriever = db.as_retriever()
retriever.invoke(query)

[Document(id='7e296df7-a8db-4a0d-8fda-999f4c5e09c6', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8c0fc355-5d1f-4c44-bf68-971a785c4da5', metadata={'source': 'speech.txt'}, page_content='…'),
 Document(id='ab17ead1-248f-45b3-9553-100fdceb9887', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German bi

In [11]:
docs = retriever.invoke(query)
docs[0].page_content 

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [12]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='7e296df7-a8db-4a0d-8fda-999f4c5e09c6', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  np.float32(10548.85)),
 (Document(id='8c0fc355-5d1f-4c44-bf68-971a785c4da5', metadata={'source': 'speech.txt'}, page_content='…'),
  np.float32(10620.179)),
 (Document(id='ab17ead1-248f-45b3-9553-100fdceb9887', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and act

In [13]:
# Can direcly pass vectors instead of docs

embedding_vector = embeddings.embed_query(query)
embedding_vector

[-1.3070794343948364,
 -1.5750443935394287,
 2.968080997467041,
 -1.1046230792999268,
 -0.37035298347473145,
 -0.2269786298274994,
 2.536571741104126,
 2.066753387451172,
 -2.0715951919555664,
 0.16810186207294464,
 -1.606722354888916,
 0.2786880135536194,
 3.0461859703063965,
 2.1982359886169434,
 -0.9383448958396912,
 0.14576749503612518,
 1.9647713899612427,
 -0.7007558941841125,
 -0.3466176688671112,
 -0.9840317368507385,
 -2.2444241046905518,
 -1.1426233053207397,
 1.3215885162353516,
 0.09400471299886703,
 1.0081614255905151,
 0.17239491641521454,
 1.797602653503418,
 -3.987805128097534,
 0.6530518531799316,
 0.11135085672140121,
 1.9845777750015259,
 1.8635845184326172,
 -0.37691763043403625,
 4.507075309753418,
 -0.8201263546943665,
 -0.16515405476093292,
 2.0671815872192383,
 0.7691006064414978,
 -1.4049654006958008,
 -1.2046531438827515,
 -0.888054370880127,
 0.16414715349674225,
 0.11054577678442001,
 1.953920841217041,
 -2.6920347213745117,
 0.636859655380249,
 0.6309411525

In [14]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score 

[Document(id='7e296df7-a8db-4a0d-8fda-999f4c5e09c6', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8c0fc355-5d1f-4c44-bf68-971a785c4da5', metadata={'source': 'speech.txt'}, page_content='…'),
 Document(id='ab17ead1-248f-45b3-9553-100fdceb9887', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German bi

In [15]:
# Saving db into local and Loading
db.save_local("faiss_index")

In [16]:
# Load 

new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
docs = new_db.similarity_search(query)

In [17]:

docs

[Document(id='7e296df7-a8db-4a0d-8fda-999f4c5e09c6', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8c0fc355-5d1f-4c44-bf68-971a785c4da5', metadata={'source': 'speech.txt'}, page_content='…'),
 Document(id='ab17ead1-248f-45b3-9553-100fdceb9887', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German bi